# Road Test

In [ ]:
import emat
emat.__version__

In [ ]:
import ema_workbench
import os, numpy, pandas, functools
from emat.util.xmle import Show
from emat.viz.scatter import scatter_graph

In [ ]:
logger = emat.util.loggers.log_to_stderr(20, True)

## Defining the Exploratory Scope

In [ ]:
road_test_scope_file = emat.package_file('model','tests','road_test.yaml')

In [ ]:
road_scope = emat.Scope(road_test_scope_file)
road_scope

A short summary of the scope can be reviewed using the `info` method.

In [ ]:
road_scope.info()

Alternatively, more detailed information about each part of the scope can be
accessed in four list attributes:

In [ ]:
road_scope.get_constants()

In [ ]:
road_scope.get_uncertainties()

In [ ]:
road_scope.get_levers()

In [ ]:
road_scope.get_measures()

## Using a Database

The exploratory modeling process will typically generate many different sets of outputs,
for different explored modeling scopes, or for different applications.  It is convenient
to organize these outputs in a database structure, so they are stored consistently and 
readily available for subsequent analysis.

The `SQLiteDB` object will create a database to store results.  When instantiated with
no arguments, the database is initialized in-memory, which will not store anything to
disk (which is convenient for this example, but in practice you will generally want to
store data to disk so that it can persist after this Python session ends).

In [ ]:
emat_db = emat.SQLiteDB()

An EMAT Scope can be stored in the database, to provide needed information about what the 
various inputs and outputs represent.

In [ ]:
road_scope.store_scope(emat_db)

Trying to store another scope with the same name (or the same scope) raises a KeyError.

In [ ]:
try:
    road_scope.store_scope(emat_db)
except KeyError as err:
    print(err)

We can review the names of scopes already stored in the database using the `read_scope_names` method.

In [ ]:
emat_db.read_scope_names()

## Experimental Design

Actually running the model can be done by the user on an *ad hoc* basis (i.e., manually defining every 
combination of inputs that will be evaluated) but the real power of EMAT comes from runnning the model
using algorithm-created experimental designs.

An important experimental design used in exploratory modeling is the Latin Hypercube.  This design selects
a random set of experiments across multiple input dimensions, to ensure "good" coverage of the 
multi-dimensional modeling space.

The `design_latin_hypercube` function creates such a design based on a `Scope`, and optionally
stores the design of experiments in a database.

In [ ]:
from emat.experiment.experimental_design import design_experiments

In [ ]:
design = design_experiments(road_scope, db=emat_db, n_samples_per_factor=10, sampler='lhs')
design.head()

In [ ]:
large_design = design_experiments(road_scope, db=emat_db, n_samples=5000, sampler='lhs', design_name='lhs_large')
large_design.head()

We can review what experimental designs have already been stored in the database using the 
`read_design_names` method of the `Database` object.

In [ ]:
emat_db.read_design_names('EMAT Road Test')

## Core Model in Python

### Model Definition

In the simplest approach for EMAT, a model can be defined as a basic Python function, which accepts all
inputs (exogenous uncertainties, policy levers, and externally defined constants) as named keyword
arguments, and returns a dictionary where the dictionary keys are names of performace measures, and 
the mapped values are the computed values for those performance measures.  The `Road_Capacity_Investment`
function provided in EMAT is an example of such a function.  This made-up example considers the 
investment in capacity expansion for a single roadway link.  The inputs to this function are described
above in the Scope, including uncertain parameters in the volume-delay function,
traffic volumes, value of travel time savings, unit construction costs, and interest rates, and policy levers including the 
amount of capacity expansion and amortization period.

In [ ]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment

In [ ]:
m = PythonCoreModel(Road_Capacity_Investment, scope=road_scope, db=emat_db)

### Model Execution

In [ ]:
from ema_workbench import SequentialEvaluator

In [ ]:
with SequentialEvaluator(m) as eval_seq:
    lhs_results = m.run_experiments(design_name='lhs', evaluator=eval_seq)
lhs_results.head()

If running a large number of experiments, it may be valuable to parallelize the 
processing using a DistributedEvaluator instead of the SequentialEvaluator.
The DistributedEvaluator uses dask.distributed to distribute the workload to
a cluster of processes, which can all be on the same machine or distributed
over multiple networked computers. (The details of using dask.distributed in 
more complex environments are beyond this scope of this example, but interested
users can refer to that package's [documentation](https://distributed.dask.org/).)

In [ ]:
large_results = m.run_experiments(design_name='lhs_large')


Once a particular design has been run once, the results can be recovered from the database without re-running the model itself.

In [ ]:
reload_results = m.read_experiments(design_name='lhs')
reload_results.head()

It is also possible to load only the parameters, or only the performance meausures.

In [ ]:
lhs_params = m.read_experiment_parameters(design_name='lhs')
lhs_params.head()

In [ ]:
lhs_outcomes = m.read_experiment_measures(design_name='lhs')
lhs_outcomes.head()

## Feature Scoring

In [ ]:
m.get_feature_scores('lhs')

## Scenario Discovery

Scenario discovery in exploratory modeling is focused on finding scenarios that are interesting to the user.  
The process generally begins through the identification of particular outcomes that are "of interest",
and the discovery process that can seek out what factor or combination of factors can result in
those outcomes.

There are a variety of methods to use for scenario discovery.  We illustrate a few here.


### PRIM

Patient rule induction method (PRIM) is an algorithm that operates on a set of data with inputs and outputs.  
It is used for locating areas of an outcome space that are of particular interest, which it does by reducing 
the data size incrementally by small amounts in an iterative process as follows:
    
- Candidate boxes are generated.  These boxes represent incrementally smaller sets of the data.  
  Each box removes a portion of the data based on the levels of a single input variable.
  * For categorical input variables, there is one box generated for each category with each box 
    removing one category from the data set.
  * For integer and continuous variables, two boxes are generated – one box that removes a 
    portion of data representing the smallest set of values for that input variable and another 
    box that removes a portion of data representing the largest set of values for that input.  
    The step size for these variables is controlled by the analyst.
- For each candidate box, the relative improvement in the number of outcomes of interest inside 
  the box is calculated and the candidate box with the highest improvement is selected.
- The data in the selected candidate box replaces the starting data and the process is repeated.

The process ends based on a stopping criteria.  For more details on the algorithm, 
see [Friedman and Fisher (1999)](http://statweb.stanford.edu/~jhf/ftp/prim.pdf) or 
[Kwakkel and Jaxa-Rozen (2016)](https://www.sciencedirect.com/science/article/pii/S1364815215301092).

The PRIM algorithm is particularly useful for scenario discovery, which broadly is the process of 
identifying particular scenarios of interest in a large and deeply uncertain dataset.   
In the context of exploratory modeling, scenario discovery is often used to obtain a better understanding 
of areas of the uncertainty space where a policy or collection of policies performs poorly because it is 
often used in tandem with robust search methods for identifying policies that perform well 
([Kwakkel and Jaxa-Rozen (2016)](https://www.sciencedirect.com/science/article/pii/S1364815215301092)).

In [ ]:
from emat.analysis import prim

In [ ]:
x = m.read_experiment_parameters(design_name='lhs_large')

In [ ]:
m.read_experiment_measures(design_name='lhs_large')

In [ ]:
prim_alg = prim.Prim(
    m.read_experiment_parameters(design_name='lhs_large'), 
    m.read_experiment_measures(design_name='lhs_large')['net_benefits']>0, 
    threshold=0.4,
)

In [ ]:
box1 = prim_alg.find_box()

In [ ]:
# This interactive version doesn't render well in documentation
# tradeoff = box1.inspect_tradeoff()
# tradeoff

In [ ]:
Show(box1.show_tradeoff())

In [ ]:
box1.inspect(45)

### CART

Classification and Regression Trees (CART) can also be used for scenario discovery. 
They partition the explored space (i.e., the scope) into a number of sections, with each partition
being added in such a way as to maximize the difference between observations on each 
side of the newly added partition divider, subject to some constraints.

In [ ]:
from ema_workbench.analysis import cart

cart_alg = cart.CART(
    m.read_experiment_parameters(design_name='lhs_large'), 
    m.read_experiment_measures(design_name='lhs_large')['net_benefits']>0,
)
cart_alg.build_tree()

In [ ]:
Show(cart_alg.show_tree(format='svg')) 

In [ ]:
cart_alg.boxes_to_dataframe(include_stats=True)

## Creating a Meta-Model

In [ ]:
mm = m.create_metamodel_from_design('lhs')
mm

To demonstrate the performance of the meta-model, we can create an
alternate design of experiments.  Note that to get different random values,
we set the `random_seed` argument to something other than the default value.

In [ ]:
design2 = design_experiments(road_scope, db=emat_db, n_samples_per_factor=10, sampler='lhs', random_seed=2)

In [ ]:
design2_results = mm.run_experiments(design2)
design2_results.head()

In [ ]:
mm.function.cross_val_scores()

### Compare Core vs Meta Model Results

We can generate a variety of plots to compare the distribution of meta-model outcomes
on the new design against the original model's results.

In [ ]:
Show(scatter_graph(
    X=[ design2_results['input_flow'],
        lhs_results['input_flow'] ],
    Y=[ design2_results['time_savings'],
        lhs_results['time_savings'],  ],
    legend_labels=[ 'meta-model',
                    'core model',  ]
))

In [ ]:
Show(scatter_graph(
    X=[ design2_results['no_build_travel_time'],
        lhs_results['no_build_travel_time'], ],
    Y=[ design2_results['time_savings'],
        lhs_results['time_savings'], ],
    legend_labels=[ 'meta-model',
                    'core model', ]
))

In [ ]:
Show(scatter_graph(
    X=[ design2_results['expand_capacity'],
        lhs_results['expand_capacity'], ],
    Y=[ design2_results['present_cost_expansion'],
        lhs_results['present_cost_expansion'], ],
    legend_labels=[ 'meta-model',
                    'core model', ]
))

## Self-Evaluating Meta-Model Performance

In [ ]:
background = design_experiments(road_scope, n_samples=1000, sampler='ulhs', db=None, random_seed=42)

In [ ]:
measure_std = mm.function.compute_std(background)

In [ ]:
measure= mm.function(background)

In [ ]:
# measure_std = background.apply(
#     lambda x: pandas.Series(mm.function.compute_std(**x)),
#     axis=1,
# )

In [ ]:
background.info()

In [ ]:
# background.join(measure_std.add_suffix("_std"))

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(
    background.join(pandas.qcut(measure.net_benefits, 5).rename("sector")),
    hue='sector',
    vars=['alpha','beta','amortization_period']
)

In [ ]:
sns.pairplot(
    background.join(measure_std.add_suffix("_std")).join(pandas.qcut(measure_std.net_benefits, 5).rename("sector")),
    hue='sector',
    vars=['alpha','beta','amortization_period']
)

In [ ]:
draws = design_experiments(road_scope, n_samples=2000, sampler='lhs', db=None, random_seed=45)

In [ ]:
drawx = mm.run_experiments(draws, db=False)

In [ ]:
drawx_std = mm.function.compute_std(draws)

In [ ]:
prim_alg = prim.Prim(
    draws, 
    drawx['net_benefits']>0, 
    threshold=0.4,
)

In [ ]:
box1 = prim_alg.find_box()

In [ ]:
tradeoff = box1.inspect_tradeoff()
tradeoff

In [ ]:
n = 31
box1.inspect(n)

In [ ]:
box1.box_lims[n]

In [ ]:
box1.select(n)

In [ ]:
box1.yi

In [ ]:
drawx_std.drop(box1.yi).net_benefits.mean()

In [ ]:
drawx_std.iloc[box1.yi].net_benefits.mean()

In [ ]:
bb = box1.to_emat_box()

In [ ]:
bb.scope = road_scope

In [ ]:
bb

In [ ]:
bb.uncertainty_thresholds

In [ ]:
bb.lever_thresholds

In [ ]:
draws_bb = design_experiments(bb, n_samples=2000, sampler='lhs', db=None, random_seed=46)

In [ ]:
draws_bb

In [ ]:
from emat.util.distributions import truncated, get_distribution_bounds
import copy

In [ ]:
def get_uncertainties(self):
    """Get a list of exogenous uncertainties."""
    result = []
    thresh = self.uncertainty_thresholds
    for i in self.scope.get_uncertainties():
        i = copy.deepcopy(i)
        if i.name in thresh:
            lowerbound, upperbound = thresh[i.name]
            if lowerbound is None:
                lowerbound = -numpy.inf
            if upperbound is None:
                upperbound = numpy.inf
            i.dist = truncated(i.dist, lowerbound, upperbound)
            i.lower_bound, i.upper_bound = get_distribution_bounds(i.dist)
        result.append(i)
    return result


In [ ]:
j = [(i, i.name, i.lower_bound, i.upper_bound) for i in get_uncertainties(bb)]
j

In [ ]:
di = j[2][0].dist

In [ ]:
di.ppf(0)

In [ ]:
di.ppf(1)

In [ ]:
di

In [ ]:
dt.ppf(0), dt.ppf(1)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
x = numpy.linspace(2,7, 1000)
plt.plot(x, di.pdf(x))
plt.plot(x, dt.pdf(x))

In [ ]:
print(road_scope.dump())